# Квазиньютоновские методы

## На прошлой лекции

- Стохастический градиент
- Стохастический градиентный спуск
- Сходимость и скорость работы
- Адаптивные методы 

## План на сегодня

- Сравнение градиентного спуска и метода Ньютона
- Построение концепции промежуточных методов
- Примеры квазиньютоновских методов
- Анализ сложности и сходимости

## Метод Ньютона и градиентный спуск

- градиентный метод получен из аппроксимации первого порядка:

$$
f_G(x) \approx f(y) + \langle f'(y), x - y \rangle + \frac{1}{2}(x-y)^{\top} \frac{1}{\alpha}I(x - y)
$$

причём при $\alpha \in (0, 1/L], f(x) \leq f_G(x)$, то есть $f_G$ - глобальная оценка $f(x)$
- метод Ньютона получен из аппроксимации второго порядка

$$
f_N(x) \approx f(y) + \langle f'(y), x - y \rangle + \frac{1}{2} (x-y)^{\top}f''(y)(x-y)
$$

## Недостатки метода Ньютона: напоминание

- необходимо хранить гессиан на каждой итерации: $O(n^2)$ памяти
- необходимо решать линейные системы: $O(n^3)$ операций
- гессиан может оказаться вырожден

## Идея квазиньютоновскизх методов

Использовать промежуточную аппроксимацию вида

$$
f_q(x) \approx f(y) + \langle f'(y), x - y \rangle + \frac{1}{2} (x-y)^{\top}{\color{red}{B(y)}}(x-y),
$$

которая даёт переход к следующей точке:

$$
x_{k+1} = x_k - \alpha_k B^{-1}_k f'(x_k) = x_k - \alpha_k H_k f'(x_k)
$$

## Как искать $B_{k+1}$?

В точке $x_{k+1}$ имеем следующую аппрокисмацию:

$$
f_q(h) \approx f(x_{k+1}) + \langle f'(x_{k+1}), h \rangle + \frac{1}{2}h^{\top}B_{k+1}h
$$

Из определения, очевидно, что $B_{k+1}$  симметричная и положительно определённая.
Какие требования естественно наложить на $f_q(h)$?

$$
f_q'(-\alpha_k h_k) = f'(x_k) \qquad f'_q(0) = f'(x_{k+1}),
$$

где первое условие даёт

$$
f'(x_{k+1}) - \alpha_k B_{k+1}h_k = f'(x_k),
$$

а второе выполняется автоматически.

### Квазиньютоновское уравнение (Secant equation)

Из первого условия получаем

$$
B_{k+1}s_k = y_k,
$$

где $s_k = x_{k+1} - x_k$ и $y_k = f'(x_{k+1}) - f'(x_k)$.

Это уравнение будет иметь решение только при $s^{\top}_k y_k > 0$. Почему?

**Вопрос:** единственным ли образом определено $B_{k+1}$?

### Как однозначно определить $B_{k+1}$?

\begin{align*}
& \min_B \| B_k - B \| \\
\text{s.t. } & B = B^{\top}\\
& Bs_k = y_k
\end{align*}

## DFP (Davidon-Fletcher-Powell)

$$
B_{k+1} = (I - \rho_k y_k s^{\top}_k)B_k(I - \rho_k s_ky^{\top}_k) + \rho_k y_k y^{\top}_k,
$$

где $\rho_k = \dfrac{1}{y^{\top}_k s_k}$ и $s_k = x_{k+1} - x_k$ и $y_k = f'(x_{k+1}) - f'(x_k)$

или с помощью формулы [Шермана-Морисона-Вудбери](https://en.wikipedia.org/wiki/Sherman%E2%80%93Morrison_formula)

$$
B^{-1}_{k+1} = H_{k+1} = H_k - \dfrac{H_ky_k y_k^{\top}H_k}{y^{\top}_kH_ky_k} + \dfrac{s_ks^{\top}_k}{y^{\top}_ks_k}
$$

**Вопрос:** какой ранг у разности матриц $B_{k+1} (H_{k+1})$ и $B_{k} (H_{k})$?

### Вывод

Общая идея квазиньютоновских методов: 

- вместо полного пересчёта гессиана на каждой итерации обновлять текущую его аппроксимацию с помощью легко вычислимого преобразования

## Метод BFGS
<img src="./bfgs.png" width=500>

**Вопрос:** какая естественная модификация метода DFP?

\begin{align*}
& \min_H \| H_k - H \| \\
\text{s.t. } & H = H^{\top}\\
& Hy_k = s_k
\end{align*}

Формула пересчёта для метода BFGS:

$$
H_{k+1} = (I - \rho_k s_ky^{\top}_k)H_k(I - \rho_k y_k s^{\top}_k) + \rho_k s_k s^{\top}_k,
$$

где $\rho_k = \dfrac{1}{y^{\top}_k s_k}$

## Сложность вычислений

- Вместо необходимости решать системы линейных уравнений за $O(n^3)$ необходимо оперировать с матрицами и векторами за $O(n^2)$. Проверьте, что вы понимаете как это делать на основе вышеприведённых формул

- Хранить полные матрицы всё ещё надо, то есть проблема с квадратичной сложностью по паимяти никуда не делась

## Идея снижения потребления памяти

- В методе BFGS нужна не сама матрица $H$, а только функция умножения её на вектор 
- Поскольку требуется локальная оценка гессиана, старые значения векторов $s$ и $y$ могут портить текущую оценку

**Идея**

- Хранить $k \ll n$ последних векторов $s$ и $y$ - снижение требуемой памяти с $n^2$ до $kn$
- Выполнение умножения на вектор рекурсивно, без явного формирования матрицы $H$ за $O(nk)$

## Метод LBFGS

- Инициализируем матрицу $H_0$, чаще всего это некоторая диагональная матрица
- Копим первые $k$ пар векторов $s$ и $y$, в процессе рекурсивно умножаем на матрицу в "распределённом виде"
- Для $k+1$ пары $(s, y)$ удаляем первую пару и обновляем матрицу $H_0$ 
- Продолжаем такие итерации до сходимости

## Умножение гессиана на вектор

```python
def get_lbfgs_direction(x):
    
    if H is None:
        
        current_grad = grad(x)
        
        return -current_grad
    
    else:
        
        q = current_grad
        
        alpha = np.zeros(len(s_hist))
        
        rho = np.zeros(len(s_hist))
        
        for i in range(len(s_hist) - 1, -1, -1):
            
            rho[i] = 1. / s_hist[i].dot(y_hist[i])
            
            alpha[i] = s_hist[i].dot(q) * rho[i]
            
            q = q - alpha[i] * y_hist[i]
            
        r = q * H
        
        for i in range(len(s_hist)):
            
            beta = rho[i] * y_hist[i].dot(r)
            
            r = r + s_hist[i] * (alpha[i] - beta)
            
    return -r

```

## Сходимость

- Квазиньютоновские методы относятся к методам, которые используют **нелинейные** преобразования градиентов
- Теоретические результаты по их сходимости довольны скудны и находятся в стадии разработки
- Экспериментально такие методы сходятся быстро и являются основными методами решения задач большой размерности при доступном **точном** градиенте
- Более формально есть результаты об их сверхлинейной сходимости
- Обобщения такого подхода на случай стохастической оценки градиента находятся в стадии исследований

## Выводы

- Приближение гессиана и аналитическое вычисление обратной матрицы снижает сложность одной итерации
- Неявное хранение матрицы и ограниченное использование истории повышвет скорость сходимости
- Обобщения с использованием стохастической оценки градиента не дают 